In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import Sampler
from PIL import Image, ImageOps
import torchvision.transforms.functional as TF
import torch.nn.functional as F
from resnest.torch import resnest50



import time
import pickle
import numpy as np
from torchvision.transforms import Lambda
import argparse
import copy
import random
import numbers
from torch.utils.tensorboard import SummaryWriter
from sklearn import metrics
import os

In [16]:
from NLBlock_MutiConv import NLBlock
from NLBlock_MutiConv import TimeConv

In [3]:
if torch.cuda.is_available()==True:
    device="cuda:2"
else:
    device ="cpu"
    
print(torch.cuda.is_available())

True


In [3]:
# Model 1  RCNet 
class M3_resnest_lstm(torch.nn.Module):
    def __init__(self):
        super(M3_resnest_lstm, self).__init__()
        resnest = resnest50(pretrained=True)
        self.share = torch.nn.Sequential()    # self.cnn = self.share
        self.share.add_module("conv1", resnest.conv1)
        self.share.add_module("bn1", resnest.bn1)
        self.share.add_module("relu", resnest.relu)
        self.share.add_module("maxpool", resnest.maxpool)
        self.share.add_module("layer1", resnest.layer1)
        self.share.add_module("layer2", resnest.layer2)
        self.share.add_module("layer3", resnest.layer3)
        self.share.add_module("layer4", resnest.layer4)
        self.share.add_module("avgpool", resnest.avgpool)
        self.dropout = nn.Dropout(p=0.2)
        self.lstm = nn.LSTM(2048, 512, batch_first=True) # feature : 512
        self.fc = nn.Linear(512, 19)      # 512 feature -> 19 classes


        init.xavier_normal_(self.lstm.all_weights[0][0])
        init.xavier_normal_(self.lstm.all_weights[0][1])
        init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        x = x.view(-1, 3, 216,216)  # 384 216 x.view(-1, 3, 224, 224) 
        x = self.share.forward(x)   # output [batchsize, 2048,1,1]
        x = x.view(-1, sequence_length, 2048) 
        
        self.lstm.flatten_parameters()
        y, _ = self.lstm(x)  # 512
        y = y.contiguous().view(-1, 512) # feature 
        y = self.dropout(y)
        y = self.fc(y)  # predict
        return y

In [16]:
M3 = M3_resnest_lstm()
LFB_path = "/media/yilin/tmrnet/models/temp/resnest/sl10_lr1e-5_6-25train/latest_model_11.pth"

In [17]:
M3.load_state_dict(torch.load(LFB_path))

<All keys matched successfully>

In [17]:
# # Model 4  resnet + lstm + non-local    
class M6_resnest_lstm_nl(torch.nn.Module):
    def __init__(self):
        super(M6_resnest_lstm_nl, self).__init__()
        resnest = resnest50(pretrained=True)
        self.share = torch.nn.Sequential()
        self.share.add_module("conv1", resnest.conv1)
        self.share.add_module("bn1", resnest.bn1)
        self.share.add_module("relu", resnest.relu)
        self.share.add_module("maxpool", resnest.maxpool)
        self.share.add_module("layer1", resnest.layer1)
        self.share.add_module("layer2", resnest.layer2)
        self.share.add_module("layer3", resnest.layer3)
        self.share.add_module("layer4", resnest.layer4)
        self.share.add_module("avgpool", resnest.avgpool)
        self.lstm = nn.LSTM(2048, 512, batch_first=True)
        self.fc_c = nn.Linear(512, 19) #7
        self.fc_h_c = nn.Linear(1024, 512)
        self.nl_block = NLBlock()
        self.dropout = nn.Dropout(p=0.5)

        init.xavier_normal_(self.lstm.all_weights[0][0])
        init.xavier_normal_(self.lstm.all_weights[0][1])
        init.xavier_uniform_(self.fc_c.weight)
        init.xavier_uniform_(self.fc_h_c.weight)

    def forward(self, x, long_feature=None):
        x = x.view(-1, 3, 216,216)
        x = self.share.forward(x)
        x = x.view(-1, sequence_length, 2048)
        self.lstm.flatten_parameters()
        y, _ = self.lstm(x)
        y = y.contiguous().view(-1, 512)
        y = y[sequence_length - 1::sequence_length]

        y_1 = self.nl_block(y, long_feature)
        y = torch.cat([y, y_1], dim=1)
        y = self.dropout(self.fc_h_c(y))
        y = F.relu(y)
        y = self.fc_c(y)
        return y

In [8]:
M6 = M6_resnest_lstm_nl()
LFB_path = "/media/yilin/pre-trained models/CataRCNet_models/resnest_non-local-only_M6.pth"

In [9]:
M6.load_state_dict(torch.load(LFB_path))

<All keys matched successfully>

In [18]:
# # Model 4  resnet + lstm + non-local    
class M9_resnest_lstm_nl_tvl(torch.nn.Module):
    def __init__(self):
        super(M9_resnest_lstm_nl_tvl, self).__init__()
        resnest = resnest50(pretrained=True)
        self.share = torch.nn.Sequential()
        self.share.add_module("conv1", resnest.conv1)
        self.share.add_module("bn1", resnest.bn1)
        self.share.add_module("relu", resnest.relu)
        self.share.add_module("maxpool", resnest.maxpool)
        self.share.add_module("layer1", resnest.layer1)
        self.share.add_module("layer2", resnest.layer2)
        self.share.add_module("layer3", resnest.layer3)
        self.share.add_module("layer4", resnest.layer4)
        self.share.add_module("avgpool", resnest.avgpool)
        self.lstm = nn.LSTM(2048, 512, batch_first=True)
        self.fc_c = nn.Linear(512, 19) #7
        self.fc_h_c = nn.Linear(1024, 512)
        self.nl_block = NLBlock()
        self.dropout = nn.Dropout(p=0.5)
        self.time_conv = TimeConv()

        init.xavier_normal_(self.lstm.all_weights[0][0])
        init.xavier_normal_(self.lstm.all_weights[0][1])
        init.xavier_uniform_(self.fc_c.weight)
        init.xavier_uniform_(self.fc_h_c.weight)

    def forward(self, x, long_feature=None):
        x = x.view(-1, 3, 216,216)
        x = self.share.forward(x)
        x = x.view(-1, sequence_length, 2048)
        self.lstm.flatten_parameters()
        y, _ = self.lstm(x)
        y = y.contiguous().view(-1, 512)
        y = y[sequence_length - 1::sequence_length]
        
        Lt = self.time_conv(long_feature)

        y_1 = self.nl_block(y, Lt)
        y = torch.cat([y, y_1], dim=1)
        y = self.dropout(self.fc_h_c(y))
        y = F.relu(y)
        y = self.fc_c(y)
        return y

In [22]:
M9 = M9_resnest_lstm_nl_tvl()
LFB_path = "/media/yilin/tmrnet/models/temp/non-local/resnest_pretrained_lr1e-5_L30_2fc_mutiConv6_6-25train/latest_model_7.pth"

In [23]:
M9.load_state_dict(torch.load(LFB_path))

<All keys matched successfully>

In [19]:
# M4
class LFB_resnest_lstm(torch.nn.Module):
    def __init__(self):
        super(LFB_resnest_lstm, self).__init__()
        resnest = resnest50(pretrained=True)
        self.share = torch.nn.Sequential()
        self.share.add_module("conv1", resnest.conv1)
        self.share.add_module("bn1", resnest.bn1)
        self.share.add_module("relu", resnest.relu)
        self.share.add_module("maxpool", resnest.maxpool)
        self.share.add_module("layer1", resnest.layer1)
        self.share.add_module("layer2", resnest.layer2)
        self.share.add_module("layer3", resnest.layer3)
        self.share.add_module("layer4", resnest.layer4)
        self.share.add_module("avgpool", resnest.avgpool)
        self.lstm = nn.LSTM(2048, 512, batch_first=True)

        init.xavier_normal_(self.lstm.all_weights[0][0])
        init.xavier_normal_(self.lstm.all_weights[0][1])

    def forward(self, x):
        x = x.view(-1, 3, 216,216)
        x = self.share.forward(x)
        x = x.view(-1, sequence_length, 2048)
        self.lstm.flatten_parameters()
        y, _ = self.lstm(x)
        y = y.contiguous().view(-1, 512)
        y = y[sequence_length - 1::sequence_length]
        return y

# LOSS FUNCTION


In [20]:
class WeightedCrossEntropy(torch.nn.Module):
    '''
    WCE
    '''       
     # 6-25
    def __init__(self, weight=torch.Tensor([0.0033, 0.4182, 0.1321, 0.0234, 0.0344, 0.0146, 0.0428, 0.0140, 0.0092,
        0.0272, 0.0096, 0.0323, 0.0341, 0.0508, 0.0151, 0.0160, 0.0365, 0.0738,
        0.0128])):
        super(WeightedCrossEntropy, self).__init__()
        
        weight = weight.to(device)
        self.weighted_cross_entropy = nn.CrossEntropyLoss(weight=weight)
        
    def forward(self, inputs, target):
        return self.weighted_cross_entropy.forward(inputs, target)

In [21]:
def pil_loader(path):
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')

class CataractsDataset(Dataset):
    def __init__(self, file_paths,file_labels, transform=None,loader=pil_loader):
        self.file_paths = file_paths
        self.file_labels_phase = file_labels[:,0]
        self.transform = transform
        self.loader = loader
        
    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, index):
        img_names = self.file_paths[index]
        labels = self.file_labels_phase[index]
        imgs = self.loader(img_names)
        if self.transform is not None:
            imgs = self.transform(imgs)

        return imgs, labels, index

In [22]:
######## center
def get_test_data(data_path):
    with open(data_path, 'rb') as f:
        test_paths_labels = pickle.load(f)

    test_paths = test_paths_labels[0]
    test_labels = test_paths_labels[1]
    test_num_each = test_paths_labels[2]

    print('test_paths   : {:6d}'.format(len(test_paths)))
    print('test_labels  : {:6d}'.format(len(test_labels)))

    test_labels = np.asarray(test_labels, dtype=np.int64)

    test_transforms = None
################################### 
    test_transforms = transforms.Compose([
            transforms.CenterCrop(216),  #transforms.RandomCrop(216),
            transforms.ToTensor()
        ])


    test_dataset = CataractsDataset(test_paths, test_labels, test_transforms)

    return test_dataset, test_num_each

# Test for M3


In [23]:
class SequenceSampler(Sampler):
    def __init__(self, data_source, idx):
        super().__init__(data_source)
        self.data_source = data_source
        self.idx = idx

    def __iter__(self):
        return iter(self.idx)

    def __len__(self):
        return len(self.idx)

In [24]:
# for sliding window
def get_start_idx(sequence_length, list_each_length):
    count = 0
    idx = []
    for i in range(len(list_each_length)):
        for j in range(count, count + (list_each_length[i] + 1 - sequence_length)):
            idx.append(j)
        count += list_each_length[i]
    return idx

In [25]:
def test_model(model,test_dataset, test_num_each):
    num_test = len(test_dataset)
    test_useful_start_idx = get_start_idx(sequence_length, test_num_each)
    test_useful_start_idx_LFB = get_start_idx(sequence_length, test_num_each)

    num_test_we_use = len(test_useful_start_idx)
    num_test_we_use_LFB = len(test_useful_start_idx_LFB)

    test_we_use_start_idx = test_useful_start_idx[0:num_test_we_use]
    test_we_use_start_idx_LFB = test_useful_start_idx_LFB[0:num_test_we_use_LFB]

    test_idx = []
    for i in range(num_test_we_use):
        for j in range(sequence_length):
            test_idx.append(test_we_use_start_idx[i] + j)

    num_test_all = len(test_idx)

    test_idx_LFB = []
    for i in range(num_test_we_use_LFB):
        for j in range(sequence_length):
            test_idx_LFB.append(test_we_use_start_idx_LFB[i] + j)

    dict_index, dict_value = zip(*list(enumerate(test_we_use_start_idx_LFB)))
    dict_test_start_idx_LFB = dict(zip(dict_value, dict_index))

    print('num test start idx : {:6d}'.format(len(test_useful_start_idx)))
    print('last idx test start: {:6d}'.format(test_useful_start_idx[-1]))
    print('num of test dataset: {:6d}'.format(num_test))
    print('num of test we use : {:6d}'.format(num_test_we_use))
    print('num of all test use: {:6d}'.format(num_test_all))
   # TODO sampler

    test_loader = DataLoader(test_dataset,
                             batch_size=test_batch_size,
                             sampler=SequenceSampler(test_dataset, test_idx),
                             num_workers=workers)

    
    torch.cuda.empty_cache()
    
    ###########################3
    # model = resnet_lstm()
    #model.load_state_dict(torch.load(model_path))
    model.load_state_dict(torch.load(LFB_path))
    
    ###############################
    

    model.to(device)

    criterion = WeightedCrossEntropy()   #size_average=False

    model.eval()
    test_loss = 0.0
    test_corrects = 0
    test_start_time = time.time()

    all_preds = []
    all_preds_score = []


    with torch.no_grad():

        for data in test_loader:
            

            torch.cuda.empty_cache()            


            inputs, labels = data[0].to(device), data[1].to(device)
            labels = labels[(sequence_length - 1)::sequence_length]


            inputs = inputs.view(-1, sequence_length, 3, 216,216)


            outputs = model.forward(inputs)

            outputs = outputs[sequence_length - 1::sequence_length]
            # softmax
            Sm = nn.Softmax()
            outputs = Sm(outputs)
            possibility, preds = torch.max(outputs.data, 1)


            for i in range(len(preds)):
                all_preds.append(preds[i].data.cpu())
            for i in range(len(possibility)):
                all_preds_score.append(possibility[i].data.cpu())
            
            
            loss = criterion(outputs, labels)


            test_loss += loss.data.item()
            test_corrects += torch.sum(preds == labels.data)
            print("test_corrects:",test_corrects)

    test_elapsed_time = time.time() - test_start_time
    test_accuracy = float(test_corrects) / float(num_test_we_use)
    test_average_loss = test_loss / num_test_we_use

    print('type of all_preds:', type(all_preds))
    print('leng of all preds:', len(all_preds))
    save_test = int("{:4.0f}".format(test_accuracy * 10000))
    pred_name = "./test_results_pkl/"+model_pure_name + '_test_' + str(save_test)  + '.pkl'
    

    with open(pred_name, 'wb') as f:
        pickle.dump(all_preds, f)

    print('test elapsed: {:2.0f}m{:2.0f}s'
          ' test loss: {:4.4f}'
          ' test accu: {:.4f}'
          .format(test_elapsed_time // 60,
                  test_elapsed_time % 60,
                  test_average_loss, test_accuracy))

In [21]:
# TEST25
test_dataset, test_num_each = get_test_data('../gen_datasets/test_paths_labels.pkl')

test_paths   :  15425
test_labels  :  15425


In [11]:

sequence_length = 10    # the length of input clip 
test_batch_size = 100  # batch size 
workers = 2   
MODEL = M3_resnest_lstm() # M2_densenet_lstm() /  M3_resnest_lstm()


#########################################
device = "cuda:0"  

model_pure_name= 'LFB-resnest'
 # 'resnet/sl10_lr5e-5_6-25train/' cuda:0
LFB_path = "/media/yilin/tmrnet/models/temp/resnest/sl10_lr1e-5_6-25train_2/latest_model_1.pth"


# test M3
test_model(MODEL, test_dataset, test_num_each)

num test start idx :  15200
last idx test start:  15415
num of test dataset:  15425
num of test we use :  15200
num of all test use: 152000


/tmp/ipykernel_2482/935845069.py:83: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = Sm(outputs)


test_corrects: tensor(7, device='cuda:3')
test_corrects: tensor(7, device='cuda:3')
test_corrects: tensor(8, device='cuda:3')
test_corrects: tensor(14, device='cuda:3')
test_corrects: tensor(19, device='cuda:3')
test_corrects: tensor(19, device='cuda:3')
test_corrects: tensor(24, device='cuda:3')
test_corrects: tensor(33, device='cuda:3')
test_corrects: tensor(37, device='cuda:3')
test_corrects: tensor(44, device='cuda:3')
test_corrects: tensor(46, device='cuda:3')
test_corrects: tensor(47, device='cuda:3')
test_corrects: tensor(47, device='cuda:3')
test_corrects: tensor(47, device='cuda:3')
test_corrects: tensor(47, device='cuda:3')
test_corrects: tensor(47, device='cuda:3')
test_corrects: tensor(47, device='cuda:3')
test_corrects: tensor(47, device='cuda:3')
test_corrects: tensor(49, device='cuda:3')
test_corrects: tensor(51, device='cuda:3')
test_corrects: tensor(55, device='cuda:3')
test_corrects: tensor(63, device='cuda:3')
test_corrects: tensor(64, device='cuda:3')
test_corrects:

test_corrects: tensor(790, device='cuda:3')
test_corrects: tensor(800, device='cuda:3')
test_corrects: tensor(806, device='cuda:3')
test_corrects: tensor(806, device='cuda:3')
test_corrects: tensor(812, device='cuda:3')
test_corrects: tensor(822, device='cuda:3')
test_corrects: tensor(831, device='cuda:3')
test_corrects: tensor(831, device='cuda:3')
test_corrects: tensor(832, device='cuda:3')
test_corrects: tensor(838, device='cuda:3')
test_corrects: tensor(847, device='cuda:3')
test_corrects: tensor(847, device='cuda:3')
test_corrects: tensor(857, device='cuda:3')
test_corrects: tensor(867, device='cuda:3')
test_corrects: tensor(877, device='cuda:3')
test_corrects: tensor(887, device='cuda:3')
test_corrects: tensor(897, device='cuda:3')
test_corrects: tensor(907, device='cuda:3')
test_corrects: tensor(917, device='cuda:3')
test_corrects: tensor(927, device='cuda:3')
test_corrects: tensor(936, device='cuda:3')
test_corrects: tensor(946, device='cuda:3')
test_corrects: tensor(948, devic

test_corrects: tensor(2208, device='cuda:3')
test_corrects: tensor(2217, device='cuda:3')
test_corrects: tensor(2227, device='cuda:3')
test_corrects: tensor(2237, device='cuda:3')
test_corrects: tensor(2246, device='cuda:3')
test_corrects: tensor(2256, device='cuda:3')
test_corrects: tensor(2266, device='cuda:3')
test_corrects: tensor(2276, device='cuda:3')
test_corrects: tensor(2286, device='cuda:3')
test_corrects: tensor(2296, device='cuda:3')
test_corrects: tensor(2302, device='cuda:3')
test_corrects: tensor(2312, device='cuda:3')
test_corrects: tensor(2322, device='cuda:3')
test_corrects: tensor(2332, device='cuda:3')
test_corrects: tensor(2342, device='cuda:3')
test_corrects: tensor(2352, device='cuda:3')
test_corrects: tensor(2357, device='cuda:3')
test_corrects: tensor(2364, device='cuda:3')
test_corrects: tensor(2373, device='cuda:3')
test_corrects: tensor(2382, device='cuda:3')
test_corrects: tensor(2390, device='cuda:3')
test_corrects: tensor(2400, device='cuda:3')
test_corre

...

test_corrects: tensor(5074, device='cuda:3')
test_corrects: tensor(5084, device='cuda:3')
test_corrects: tensor(5094, device='cuda:3')
test_corrects: tensor(5104, device='cuda:3')
test_corrects: tensor(5114, device='cuda:3')
test_corrects: tensor(5121, device='cuda:3')
test_corrects: tensor(5128, device='cuda:3')
test_corrects: tensor(5138, device='cuda:3')
test_corrects: tensor(5148, device='cuda:3')
test_corrects: tensor(5158, device='cuda:3')
test_corrects: tensor(5168, device='cuda:3')
test_corrects: tensor(5178, device='cuda:3')
test_corrects: tensor(5188, device='cuda:3')
test_corrects: tensor(5198, device='cuda:3')
test_corrects: tensor(5208, device='cuda:3')
test_corrects: tensor(5217, device='cuda:3')
test_corrects: tensor(5224, device='cuda:3')
test_corrects: tensor(5231, device='cuda:3')
test_corrects: tensor(5237, device='cuda:3')
test_corrects: tensor(5247, device='cuda:3')
test_corrects: tensor(5256, device='cuda:3')
test_corrects: tensor(5265, device='cuda:3')
test_corre

test_corrects: tensor(6597, device='cuda:3')
test_corrects: tensor(6607, device='cuda:3')
test_corrects: tensor(6617, device='cuda:3')
test_corrects: tensor(6627, device='cuda:3')
test_corrects: tensor(6637, device='cuda:3')
test_corrects: tensor(6646, device='cuda:3')
test_corrects: tensor(6656, device='cuda:3')
test_corrects: tensor(6666, device='cuda:3')
test_corrects: tensor(6676, device='cuda:3')
test_corrects: tensor(6686, device='cuda:3')
test_corrects: tensor(6696, device='cuda:3')
test_corrects: tensor(6706, device='cuda:3')
test_corrects: tensor(6716, device='cuda:3')
test_corrects: tensor(6726, device='cuda:3')
test_corrects: tensor(6736, device='cuda:3')
test_corrects: tensor(6746, device='cuda:3')
test_corrects: tensor(6755, device='cuda:3')
test_corrects: tensor(6765, device='cuda:3')
test_corrects: tensor(6765, device='cuda:3')
test_corrects: tensor(6771, device='cuda:3')
test_corrects: tensor(6777, device='cuda:3')
test_corrects: tensor(6786, device='cuda:3')
test_corre

test_corrects: tensor(8069, device='cuda:3')
test_corrects: tensor(8079, device='cuda:3')
test_corrects: tensor(8089, device='cuda:3')
test_corrects: tensor(8099, device='cuda:3')
test_corrects: tensor(8109, device='cuda:3')
test_corrects: tensor(8116, device='cuda:3')
test_corrects: tensor(8126, device='cuda:3')
test_corrects: tensor(8136, device='cuda:3')
test_corrects: tensor(8145, device='cuda:3')
test_corrects: tensor(8150, device='cuda:3')
test_corrects: tensor(8154, device='cuda:3')
test_corrects: tensor(8164, device='cuda:3')
test_corrects: tensor(8174, device='cuda:3')
test_corrects: tensor(8184, device='cuda:3')
test_corrects: tensor(8193, device='cuda:3')
test_corrects: tensor(8202, device='cuda:3')
test_corrects: tensor(8210, device='cuda:3')
test_corrects: tensor(8220, device='cuda:3')
test_corrects: tensor(8230, device='cuda:3')
test_corrects: tensor(8240, device='cuda:3')
test_corrects: tensor(8250, device='cuda:3')
test_corrects: tensor(8260, device='cuda:3')
test_corre

test_corrects: tensor(9149, device='cuda:3')
test_corrects: tensor(9158, device='cuda:3')
test_corrects: tensor(9168, device='cuda:3')
test_corrects: tensor(9177, device='cuda:3')
test_corrects: tensor(9187, device='cuda:3')
test_corrects: tensor(9197, device='cuda:3')
test_corrects: tensor(9206, device='cuda:3')
test_corrects: tensor(9212, device='cuda:3')
test_corrects: tensor(9220, device='cuda:3')
test_corrects: tensor(9225, device='cuda:3')
test_corrects: tensor(9233, device='cuda:3')
test_corrects: tensor(9243, device='cuda:3')
test_corrects: tensor(9252, device='cuda:3')
test_corrects: tensor(9258, device='cuda:3')
test_corrects: tensor(9268, device='cuda:3')
test_corrects: tensor(9278, device='cuda:3')
test_corrects: tensor(9288, device='cuda:3')
test_corrects: tensor(9298, device='cuda:3')
test_corrects: tensor(9308, device='cuda:3')
test_corrects: tensor(9315, device='cuda:3')
test_corrects: tensor(9321, device='cuda:3')
test_corrects: tensor(9321, device='cuda:3')
test_corre

test_corrects: tensor(10671, device='cuda:3')
test_corrects: tensor(10675, device='cuda:3')
test_corrects: tensor(10685, device='cuda:3')
test_corrects: tensor(10694, device='cuda:3')
test_corrects: tensor(10696, device='cuda:3')
test_corrects: tensor(10706, device='cuda:3')
test_corrects: tensor(10716, device='cuda:3')
test_corrects: tensor(10726, device='cuda:3')
test_corrects: tensor(10736, device='cuda:3')
test_corrects: tensor(10746, device='cuda:3')
test_corrects: tensor(10756, device='cuda:3')
test_corrects: tensor(10766, device='cuda:3')
test_corrects: tensor(10775, device='cuda:3')
test_corrects: tensor(10785, device='cuda:3')
test_corrects: tensor(10795, device='cuda:3')
test_corrects: tensor(10805, device='cuda:3')
test_corrects: tensor(10815, device='cuda:3')
test_corrects: tensor(10825, device='cuda:3')
test_corrects: tensor(10835, device='cuda:3')
test_corrects: tensor(10845, device='cuda:3')
test_corrects: tensor(10855, device='cuda:3')
test_corrects: tensor(10865, devic

# TESTM6 M9

In [26]:
# Long Term Feature bank
g_LFB_train = np.zeros(shape=(0, 512))
g_LFB_val = np.zeros(shape=(0, 512))

In [27]:
def get_long_feature(start_index_list, dict_start_idx_LFB, lfb):
    long_feature = []
    for j in range(len(start_index_list)):
        long_feature_each = []
        
        # 上一个存在feature的index
        last_LFB_index_no_empty = dict_start_idx_LFB[int(start_index_list[j])]
        
        # 
        for k in range(LFB_length):
            LFB_index = (start_index_list[j] - k - 1)
            if int(LFB_index) in dict_start_idx_LFB:                
                LFB_index = dict_start_idx_LFB[int(LFB_index)]
                long_feature_each.append(lfb[LFB_index])
                last_LFB_index_no_empty = LFB_index
            else:
                long_feature_each.append(lfb[last_LFB_index_no_empty])
            
        long_feature.append(long_feature_each)
    return long_feature

In [28]:
### m6 m7
g_LFB_test = np.zeros(shape=(0, 512))

def test_model(model,test_dataset, test_num_each):
    num_test = len(test_dataset)
    test_useful_start_idx = get_start_idx(sequence_length, test_num_each)
    test_useful_start_idx_LFB = get_start_idx(sequence_length, test_num_each)

    num_test_we_use = len(test_useful_start_idx)
    num_test_we_use_LFB = len(test_useful_start_idx_LFB)

    test_we_use_start_idx = test_useful_start_idx[0:num_test_we_use]
    test_we_use_start_idx_LFB = test_useful_start_idx_LFB[0:num_test_we_use_LFB]

    test_idx = []
    for i in range(num_test_we_use):
        for j in range(sequence_length):
            test_idx.append(test_we_use_start_idx[i] + j)

    num_test_all = len(test_idx)

    test_idx_LFB = []
    for i in range(num_test_we_use_LFB):
        for j in range(sequence_length):
            test_idx_LFB.append(test_we_use_start_idx_LFB[i] + j)

    dict_index, dict_value = zip(*list(enumerate(test_we_use_start_idx_LFB)))
    dict_test_start_idx_LFB = dict(zip(dict_value, dict_index))

    print('num test start idx : {:6d}'.format(len(test_useful_start_idx)))
    print('last idx test start: {:6d}'.format(test_useful_start_idx[-1]))
    print('num of test dataset: {:6d}'.format(num_test))
    print('num of test we use : {:6d}'.format(num_test_we_use))
    print('num of all test use: {:6d}'.format(num_test_all))
# TODO sampler

    test_loader = DataLoader(test_dataset,
                             batch_size=test_batch_size,
                             sampler=SequenceSampler(test_dataset, test_idx),
                             num_workers=workers)

    global g_LFB_test
    print("loading features!>.........")

    if not load_exist_LFB:
        g_LFB_test = np.zeros(shape=(0, 512))

        test_feature_loader = DataLoader(
            test_dataset,
            batch_size=test_batch_size,
            sampler=SequenceSampler(test_dataset, test_idx_LFB),
            num_workers=workers,
            pin_memory=False
        )

        ####################
        # model_LFB = resnet_lstm_LFB()

        model_LFB.load_state_dict(torch.load(LFB_path), strict=False)        
        ########################

        model_LFB.to(device)

        for params in model_LFB.parameters():
            params.requires_grad = False

        model_LFB.eval()

        with torch.no_grad():

            for data in test_feature_loader:

                inputs, labels_phase = data[0].to(device), data[1].to(device)
                

                inputs = inputs.view(-1, sequence_length, 3, 216,216)  #
                outputs_feature = model_LFB.forward(inputs)

                for j in range(len(outputs_feature)):
                    save_feature = outputs_feature.data.cpu()[j].numpy()
                    save_feature = save_feature.reshape(1, 512)
                    g_LFB_test = np.concatenate((g_LFB_test, save_feature), axis=0)

                print("test feature length:", len(g_LFB_test))

        print("finish!")
        g_LFB_test = np.array(g_LFB_test)
        
#         LFB_train_path_save_path = "./LFB/LFB_test_resnet.pkl"

        with open(LFB_test_path_save_path, 'wb') as f: 
            #"../Training TMRNet/LFB/g_LFB_test.pkl"
            # with open("./LFB/g_LFB_test.pkl", 'wb') as f:
            pickle.dump(g_LFB_test, f)

    else:
        with open(LFB_test_path_save_path, 'rb') as f: 
            g_LFB_test = pickle.load(f)

        print("load completed")

    print("g_LFB_test shape:", g_LFB_test.shape)    
    
        
    
    torch.cuda.empty_cache()
    
    ###########################3
    # model = resnet_lstm()
    #model.load_state_dict(torch.load(model_path))
    model.load_state_dict(torch.load(model_path))
    
    ###############################
    

    model.to(device)

    criterion = WeightedCrossEntropy()   #size_average=False

    model.eval()
    test_loss = 0.0
    test_corrects = 0
    test_start_time = time.time()

    all_preds = []
    all_preds_score = []


    with torch.no_grad():

        for data in test_loader:
            
            torch.cuda.empty_cache()            


            inputs, labels = data[0].to(device), data[1].to(device)
            labels = labels[(sequence_length - 1)::sequence_length]


            start_index_list = data[2]
            start_index_list = start_index_list[0::sequence_length]
            long_feature = get_long_feature(start_index_list=start_index_list,
                                            dict_start_idx_LFB=dict_test_start_idx_LFB,
                                            lfb=g_LFB_test)
            long_feature = torch.Tensor(long_feature).to(device)

            inputs = inputs.view(-1, sequence_length, 3, 216,216)

            outputs = model.forward(inputs, long_feature=long_feature)

            # softmax
            Sm = nn.Softmax()
            outputs = Sm(outputs)
            possibility, preds = torch.max(outputs.data, 1)


            for i in range(len(preds)):
                all_preds.append(preds[i].data.cpu())
            for i in range(len(possibility)):
                all_preds_score.append(possibility[i].data.cpu())
            
            
            loss = criterion(outputs, labels)


            test_loss += loss.data.item()
            test_corrects += torch.sum(preds == labels.data)
            print("test_corrects:",test_corrects)

    test_elapsed_time = time.time() - test_start_time
    test_accuracy = float(test_corrects) / float(num_test_we_use)
    test_average_loss = test_loss / num_test_we_use

    print('type of all_preds:', type(all_preds))
    print('leng of all preds:', len(all_preds))
    save_test = int("{:4.0f}".format(test_accuracy * 10000))
    pred_name = "./test_results_pkl/"+model_pure_name + '_test_' + str(save_test)  + '.pkl'
    

    with open(pred_name, 'wb') as f:
        pickle.dump(all_preds, f)

    print('test elapsed: {:2.0f}m{:2.0f}s'
          ' test loss: {:4.4f}'
          ' test accu: {:.4f}'
          .format(test_elapsed_time // 60,
                  test_elapsed_time % 60,
                  test_average_loss, test_accuracy))



In [29]:
test_dataset, test_num_each = get_test_data('../gen_datasets/test_paths_labels.pkl')

test_paths   :  15425
test_labels  :  15425


In [48]:
len(test_num_each)

25

In [27]:
# the first time to test, it is necessary to load feature bank

sequence_length = 10    # the length of input clip 
test_batch_size = 100  # batch size 
workers = 2
LFB_length = 30
load_exist_LFB = False # False

MODEL = M6_resnest_lstm_nl() # M5_densenet_lstm_nl() /  M6_resnest_lstm_nl()


model_LFB = LFB_resnest_lstm()
# long feature bank path
LFB_test_path_save_path = "./LFB/LFB_test_resnest.pkl"




#########################################
device = "cuda:0"  
model_pure_name= 'NL-resnest' # used for the name of result pkl file
# test model path
model_path = "/media/yilin/pre-trained models/CataRCNet_models/resnest_non-local-only_M6.pth"

# lfb model path
LFB_path = "/media/yilin/tmrnet/models/temp/resnest/sl10_lr1e-5_6-25train_2/latest_model_1.pth"
###########################################


# train M4  # just example output using test01-test02
test_model(MODEL,test_dataset, test_num_each)

num test start idx :  15200
last idx test start:  15415
num of test dataset:  15425
num of test we use :  15200
num of all test use: 152000
loading features!>.........
load completed
g_LFB_test shape: (15200, 512)


/tmp/ipykernel_9146/201459335.py:153: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = Sm(outputs)


test_corrects: tensor(10, device='cuda:0')
test_corrects: tensor(16, device='cuda:0')
test_corrects: tensor(16, device='cuda:0')
test_corrects: tensor(18, device='cuda:0')
test_corrects: tensor(23, device='cuda:0')
test_corrects: tensor(23, device='cuda:0')
test_corrects: tensor(31, device='cuda:0')
test_corrects: tensor(41, device='cuda:0')
test_corrects: tensor(44, device='cuda:0')
test_corrects: tensor(51, device='cuda:0')
test_corrects: tensor(53, device='cuda:0')
test_corrects: tensor(54, device='cuda:0')
test_corrects: tensor(54, device='cuda:0')
test_corrects: tensor(54, device='cuda:0')
test_corrects: tensor(54, device='cuda:0')
test_corrects: tensor(54, device='cuda:0')
test_corrects: tensor(56, device='cuda:0')
test_corrects: tensor(56, device='cuda:0')
test_corrects: tensor(56, device='cuda:0')
test_corrects: tensor(56, device='cuda:0')
test_corrects: tensor(57, device='cuda:0')
test_corrects: tensor(67, device='cuda:0')
test_corrects: tensor(77, device='cuda:0')
test_correc

test_corrects: tensor(733, device='cuda:0')
test_corrects: tensor(740, device='cuda:0')
test_corrects: tensor(744, device='cuda:0')
test_corrects: tensor(744, device='cuda:0')
test_corrects: tensor(750, device='cuda:0')
test_corrects: tensor(760, device='cuda:0')
test_corrects: tensor(770, device='cuda:0')
test_corrects: tensor(770, device='cuda:0')
test_corrects: tensor(774, device='cuda:0')
test_corrects: tensor(777, device='cuda:0')
test_corrects: tensor(786, device='cuda:0')
test_corrects: tensor(786, device='cuda:0')
test_corrects: tensor(796, device='cuda:0')
test_corrects: tensor(806, device='cuda:0')
test_corrects: tensor(816, device='cuda:0')
test_corrects: tensor(826, device='cuda:0')
test_corrects: tensor(836, device='cuda:0')
test_corrects: tensor(846, device='cuda:0')
test_corrects: tensor(856, device='cuda:0')
test_corrects: tensor(866, device='cuda:0')
test_corrects: tensor(876, device='cuda:0')
test_corrects: tensor(886, device='cuda:0')
test_corrects: tensor(891, devic

test_corrects: tensor(2083, device='cuda:0')
test_corrects: tensor(2091, device='cuda:0')
test_corrects: tensor(2101, device='cuda:0')
test_corrects: tensor(2111, device='cuda:0')
test_corrects: tensor(2121, device='cuda:0')
test_corrects: tensor(2130, device='cuda:0')
test_corrects: tensor(2140, device='cuda:0')
test_corrects: tensor(2150, device='cuda:0')
test_corrects: tensor(2160, device='cuda:0')
test_corrects: tensor(2170, device='cuda:0')
test_corrects: tensor(2180, device='cuda:0')
test_corrects: tensor(2185, device='cuda:0')
test_corrects: tensor(2195, device='cuda:0')
test_corrects: tensor(2205, device='cuda:0')
test_corrects: tensor(2215, device='cuda:0')
test_corrects: tensor(2225, device='cuda:0')
test_corrects: tensor(2235, device='cuda:0')
test_corrects: tensor(2245, device='cuda:0')
test_corrects: tensor(2253, device='cuda:0')
test_corrects: tensor(2262, device='cuda:0')
test_corrects: tensor(2271, device='cuda:0')
test_corrects: tensor(2279, device='cuda:0')
test_corre

...

test_corrects: tensor(4704, device='cuda:0')
test_corrects: tensor(4713, device='cuda:0')
test_corrects: tensor(4714, device='cuda:0')
test_corrects: tensor(4715, device='cuda:0')
test_corrects: tensor(4719, device='cuda:0')
test_corrects: tensor(4728, device='cuda:0')
test_corrects: tensor(4738, device='cuda:0')
test_corrects: tensor(4745, device='cuda:0')
test_corrects: tensor(4749, device='cuda:0')
test_corrects: tensor(4759, device='cuda:0')
test_corrects: tensor(4769, device='cuda:0')
test_corrects: tensor(4779, device='cuda:0')
test_corrects: tensor(4789, device='cuda:0')
test_corrects: tensor(4799, device='cuda:0')
test_corrects: tensor(4808, device='cuda:0')
test_corrects: tensor(4817, device='cuda:0')
test_corrects: tensor(4819, device='cuda:0')
test_corrects: tensor(4827, device='cuda:0')
test_corrects: tensor(4835, device='cuda:0')
test_corrects: tensor(4844, device='cuda:0')
test_corrects: tensor(4851, device='cuda:0')
test_corrects: tensor(4861, device='cuda:0')
test_corre

test_corrects: tensor(6198, device='cuda:0')
test_corrects: tensor(6204, device='cuda:0')
test_corrects: tensor(6205, device='cuda:0')
test_corrects: tensor(6205, device='cuda:0')
test_corrects: tensor(6205, device='cuda:0')
test_corrects: tensor(6205, device='cuda:0')
test_corrects: tensor(6207, device='cuda:0')
test_corrects: tensor(6217, device='cuda:0')
test_corrects: tensor(6224, device='cuda:0')
test_corrects: tensor(6234, device='cuda:0')
test_corrects: tensor(6242, device='cuda:0')
test_corrects: tensor(6252, device='cuda:0')
test_corrects: tensor(6262, device='cuda:0')
test_corrects: tensor(6272, device='cuda:0')
test_corrects: tensor(6282, device='cuda:0')
test_corrects: tensor(6292, device='cuda:0')
test_corrects: tensor(6302, device='cuda:0')
test_corrects: tensor(6312, device='cuda:0')
test_corrects: tensor(6322, device='cuda:0')
test_corrects: tensor(6330, device='cuda:0')
test_corrects: tensor(6340, device='cuda:0')
test_corrects: tensor(6347, device='cuda:0')
test_corre

test_corrects: tensor(7631, device='cuda:0')
test_corrects: tensor(7641, device='cuda:0')
test_corrects: tensor(7649, device='cuda:0')
test_corrects: tensor(7659, device='cuda:0')
test_corrects: tensor(7669, device='cuda:0')
test_corrects: tensor(7676, device='cuda:0')
test_corrects: tensor(7686, device='cuda:0')
test_corrects: tensor(7696, device='cuda:0')
test_corrects: tensor(7706, device='cuda:0')
test_corrects: tensor(7716, device='cuda:0')
test_corrects: tensor(7726, device='cuda:0')
test_corrects: tensor(7735, device='cuda:0')
test_corrects: tensor(7744, device='cuda:0')
test_corrects: tensor(7754, device='cuda:0')
test_corrects: tensor(7764, device='cuda:0')
test_corrects: tensor(7771, device='cuda:0')
test_corrects: tensor(7775, device='cuda:0')
test_corrects: tensor(7785, device='cuda:0')
test_corrects: tensor(7789, device='cuda:0')
test_corrects: tensor(7798, device='cuda:0')
test_corrects: tensor(7808, device='cuda:0')
test_corrects: tensor(7818, device='cuda:0')
test_corre

test_corrects: tensor(8797, device='cuda:0')
test_corrects: tensor(8806, device='cuda:0')
test_corrects: tensor(8815, device='cuda:0')
test_corrects: tensor(8824, device='cuda:0')
test_corrects: tensor(8834, device='cuda:0')
test_corrects: tensor(8844, device='cuda:0')
test_corrects: tensor(8853, device='cuda:0')
test_corrects: tensor(8863, device='cuda:0')
test_corrects: tensor(8873, device='cuda:0')
test_corrects: tensor(8882, device='cuda:0')
test_corrects: tensor(8887, device='cuda:0')
test_corrects: tensor(8896, device='cuda:0')
test_corrects: tensor(8902, device='cuda:0')
test_corrects: tensor(8909, device='cuda:0')
test_corrects: tensor(8914, device='cuda:0')
test_corrects: tensor(8918, device='cuda:0')
test_corrects: tensor(8927, device='cuda:0')
test_corrects: tensor(8937, device='cuda:0')
test_corrects: tensor(8947, device='cuda:0')
test_corrects: tensor(8957, device='cuda:0')
test_corrects: tensor(8967, device='cuda:0')
test_corrects: tensor(8973, device='cuda:0')
test_corre

test_corrects: tensor(10235, device='cuda:0')
test_corrects: tensor(10236, device='cuda:0')
test_corrects: tensor(10236, device='cuda:0')
test_corrects: tensor(10237, device='cuda:0')
test_corrects: tensor(10241, device='cuda:0')
test_corrects: tensor(10245, device='cuda:0')
test_corrects: tensor(10248, device='cuda:0')
test_corrects: tensor(10248, device='cuda:0')
test_corrects: tensor(10257, device='cuda:0')
test_corrects: tensor(10267, device='cuda:0')
test_corrects: tensor(10277, device='cuda:0')
test_corrects: tensor(10287, device='cuda:0')
test_corrects: tensor(10297, device='cuda:0')
test_corrects: tensor(10307, device='cuda:0')
test_corrects: tensor(10316, device='cuda:0')
test_corrects: tensor(10325, device='cuda:0')
test_corrects: tensor(10335, device='cuda:0')
test_corrects: tensor(10341, device='cuda:0')
test_corrects: tensor(10341, device='cuda:0')
test_corrects: tensor(10347, device='cuda:0')
test_corrects: tensor(10352, device='cuda:0')
test_corrects: tensor(10359, devic

In [30]:
# the first time to test, it is necessary to load feature bank

sequence_length = 10    # the length of input clip 
test_batch_size = 100  # batch size 
workers = 2
LFB_length = 30
load_exist_LFB = True # False

MODEL = M6_resnest_lstm_nl() # M5_densenet_lstm_nl() /  M6_resnest_lstm_nl()


model_LFB = LFB_resnest_lstm()
# long feature bank path
LFB_test_path_save_path = "./LFB/LFB_test_resnest.pkl"




#########################################
device = "cuda:0"  
model_pure_name= 'NL-resnest' # used for the name of result pkl file
# test model path
model_path = "/media/yilin/tmrnet/models/temp/non-local/resnest_pretrained_only_lr1e-5_L30_2fc_6-25train/latest_model_11.pth"

# lfb model path
LFB_path = "/media/yilin/tmrnet/models/temp/resnest/sl10_lr1e-5_6-25train_2/latest_model_1.pth"
###########################################


# train M4  # just example output using test01-test02
test_model(MODEL,test_dataset, test_num_each)

num test start idx :  15200
last idx test start:  15415
num of test dataset:  15425
num of test we use :  15200
num of all test use: 152000
loading features!>.........
load completed
g_LFB_test shape: (15200, 512)


/tmp/ipykernel_15977/201459335.py:153: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = Sm(outputs)


test_corrects: tensor(9, device='cuda:0')
test_corrects: tensor(14, device='cuda:0')
test_corrects: tensor(14, device='cuda:0')
test_corrects: tensor(20, device='cuda:0')
test_corrects: tensor(27, device='cuda:0')
test_corrects: tensor(27, device='cuda:0')
test_corrects: tensor(34, device='cuda:0')
test_corrects: tensor(44, device='cuda:0')
test_corrects: tensor(51, device='cuda:0')
test_corrects: tensor(59, device='cuda:0')
test_corrects: tensor(61, device='cuda:0')
test_corrects: tensor(62, device='cuda:0')
test_corrects: tensor(62, device='cuda:0')
test_corrects: tensor(62, device='cuda:0')
test_corrects: tensor(62, device='cuda:0')
test_corrects: tensor(62, device='cuda:0')
test_corrects: tensor(62, device='cuda:0')
test_corrects: tensor(62, device='cuda:0')
test_corrects: tensor(67, device='cuda:0')
test_corrects: tensor(72, device='cuda:0')
test_corrects: tensor(81, device='cuda:0')
test_corrects: tensor(90, device='cuda:0')
test_corrects: tensor(99, device='cuda:0')
test_correct

test_corrects: tensor(805, device='cuda:0')
test_corrects: tensor(807, device='cuda:0')
test_corrects: tensor(816, device='cuda:0')
test_corrects: tensor(821, device='cuda:0')
test_corrects: tensor(821, device='cuda:0')
test_corrects: tensor(827, device='cuda:0')
test_corrects: tensor(837, device='cuda:0')
test_corrects: tensor(846, device='cuda:0')
test_corrects: tensor(846, device='cuda:0')
test_corrects: tensor(850, device='cuda:0')
test_corrects: tensor(854, device='cuda:0')
test_corrects: tensor(863, device='cuda:0')
test_corrects: tensor(863, device='cuda:0')
test_corrects: tensor(873, device='cuda:0')
test_corrects: tensor(883, device='cuda:0')
test_corrects: tensor(893, device='cuda:0')
test_corrects: tensor(903, device='cuda:0')
test_corrects: tensor(913, device='cuda:0')
test_corrects: tensor(922, device='cuda:0')
test_corrects: tensor(929, device='cuda:0')
test_corrects: tensor(939, device='cuda:0')
test_corrects: tensor(947, device='cuda:0')
test_corrects: tensor(957, devic

test_corrects: tensor(2215, device='cuda:0')
test_corrects: tensor(2222, device='cuda:0')
test_corrects: tensor(2231, device='cuda:0')
test_corrects: tensor(2241, device='cuda:0')
test_corrects: tensor(2251, device='cuda:0')
test_corrects: tensor(2260, device='cuda:0')
test_corrects: tensor(2270, device='cuda:0')
test_corrects: tensor(2280, device='cuda:0')
test_corrects: tensor(2290, device='cuda:0')
test_corrects: tensor(2300, device='cuda:0')
test_corrects: tensor(2310, device='cuda:0')
test_corrects: tensor(2315, device='cuda:0')
test_corrects: tensor(2325, device='cuda:0')
test_corrects: tensor(2335, device='cuda:0')
test_corrects: tensor(2345, device='cuda:0')
test_corrects: tensor(2355, device='cuda:0')
test_corrects: tensor(2364, device='cuda:0')
test_corrects: tensor(2372, device='cuda:0')
test_corrects: tensor(2380, device='cuda:0')
test_corrects: tensor(2389, device='cuda:0')
test_corrects: tensor(2398, device='cuda:0')
test_corrects: tensor(2407, device='cuda:0')
test_corre

...

test_corrects: tensor(5075, device='cuda:0')
test_corrects: tensor(5084, device='cuda:0')
test_corrects: tensor(5094, device='cuda:0')
test_corrects: tensor(5101, device='cuda:0')
test_corrects: tensor(5111, device='cuda:0')
test_corrects: tensor(5121, device='cuda:0')
test_corrects: tensor(5130, device='cuda:0')
test_corrects: tensor(5140, device='cuda:0')
test_corrects: tensor(5147, device='cuda:0')
test_corrects: tensor(5154, device='cuda:0')
test_corrects: tensor(5164, device='cuda:0')
test_corrects: tensor(5174, device='cuda:0')
test_corrects: tensor(5184, device='cuda:0')
test_corrects: tensor(5194, device='cuda:0')
test_corrects: tensor(5204, device='cuda:0')
test_corrects: tensor(5214, device='cuda:0')
test_corrects: tensor(5224, device='cuda:0')
test_corrects: tensor(5234, device='cuda:0')
test_corrects: tensor(5242, device='cuda:0')
test_corrects: tensor(5248, device='cuda:0')
test_corrects: tensor(5257, device='cuda:0')
test_corrects: tensor(5265, device='cuda:0')
test_corre

test_corrects: tensor(6540, device='cuda:0')
test_corrects: tensor(6550, device='cuda:0')
test_corrects: tensor(6560, device='cuda:0')
test_corrects: tensor(6570, device='cuda:0')
test_corrects: tensor(6580, device='cuda:0')
test_corrects: tensor(6590, device='cuda:0')
test_corrects: tensor(6600, device='cuda:0')
test_corrects: tensor(6610, device='cuda:0')
test_corrects: tensor(6619, device='cuda:0')
test_corrects: tensor(6622, device='cuda:0')
test_corrects: tensor(6623, device='cuda:0')
test_corrects: tensor(6633, device='cuda:0')
test_corrects: tensor(6643, device='cuda:0')
test_corrects: tensor(6653, device='cuda:0')
test_corrects: tensor(6663, device='cuda:0')
test_corrects: tensor(6673, device='cuda:0')
test_corrects: tensor(6683, device='cuda:0')
test_corrects: tensor(6693, device='cuda:0')
test_corrects: tensor(6703, device='cuda:0')
test_corrects: tensor(6712, device='cuda:0')
test_corrects: tensor(6722, device='cuda:0')
test_corrects: tensor(6725, device='cuda:0')
test_corre

test_corrects: tensor(8038, device='cuda:0')
test_corrects: tensor(8048, device='cuda:0')
test_corrects: tensor(8058, device='cuda:0')
test_corrects: tensor(8067, device='cuda:0')
test_corrects: tensor(8077, device='cuda:0')
test_corrects: tensor(8087, device='cuda:0')
test_corrects: tensor(8097, device='cuda:0')
test_corrects: tensor(8107, device='cuda:0')
test_corrects: tensor(8117, device='cuda:0')
test_corrects: tensor(8122, device='cuda:0')
test_corrects: tensor(8128, device='cuda:0')
test_corrects: tensor(8138, device='cuda:0')
test_corrects: tensor(8147, device='cuda:0')
test_corrects: tensor(8156, device='cuda:0')
test_corrects: tensor(8166, device='cuda:0')
test_corrects: tensor(8176, device='cuda:0')
test_corrects: tensor(8186, device='cuda:0')
test_corrects: tensor(8193, device='cuda:0')
test_corrects: tensor(8201, device='cuda:0')
test_corrects: tensor(8209, device='cuda:0')
test_corrects: tensor(8219, device='cuda:0')
test_corrects: tensor(8229, device='cuda:0')
test_corre

test_corrects: tensor(9193, device='cuda:0')
test_corrects: tensor(9203, device='cuda:0')
test_corrects: tensor(9213, device='cuda:0')
test_corrects: tensor(9223, device='cuda:0')
test_corrects: tensor(9230, device='cuda:0')
test_corrects: tensor(9238, device='cuda:0')
test_corrects: tensor(9248, device='cuda:0')
test_corrects: tensor(9257, device='cuda:0')
test_corrects: tensor(9267, device='cuda:0')
test_corrects: tensor(9277, device='cuda:0')
test_corrects: tensor(9285, device='cuda:0')
test_corrects: tensor(9291, device='cuda:0')
test_corrects: tensor(9299, device='cuda:0')
test_corrects: tensor(9305, device='cuda:0')
test_corrects: tensor(9315, device='cuda:0')
test_corrects: tensor(9325, device='cuda:0')
test_corrects: tensor(9333, device='cuda:0')
test_corrects: tensor(9338, device='cuda:0')
test_corrects: tensor(9348, device='cuda:0')
test_corrects: tensor(9358, device='cuda:0')
test_corrects: tensor(9368, device='cuda:0')
test_corrects: tensor(9378, device='cuda:0')
test_corre

test_corrects: tensor(10763, device='cuda:0')
test_corrects: tensor(10768, device='cuda:0')
test_corrects: tensor(10772, device='cuda:0')
test_corrects: tensor(10777, device='cuda:0')
test_corrects: tensor(10777, device='cuda:0')
test_corrects: tensor(10781, device='cuda:0')
test_corrects: tensor(10791, device='cuda:0')
test_corrects: tensor(10800, device='cuda:0')
test_corrects: tensor(10808, device='cuda:0')
test_corrects: tensor(10818, device='cuda:0')
test_corrects: tensor(10828, device='cuda:0')
test_corrects: tensor(10838, device='cuda:0')
test_corrects: tensor(10848, device='cuda:0')
test_corrects: tensor(10856, device='cuda:0')
test_corrects: tensor(10866, device='cuda:0')
test_corrects: tensor(10876, device='cuda:0')
test_corrects: tensor(10885, device='cuda:0')
test_corrects: tensor(10895, device='cuda:0')
test_corrects: tensor(10905, device='cuda:0')
test_corrects: tensor(10914, device='cuda:0')
test_corrects: tensor(10924, device='cuda:0')
test_corrects: tensor(10933, devic

In [31]:
# the first time to test, it is necessary to load feature bank

sequence_length = 10    # the length of input clip 
test_batch_size = 100  # batch size 
workers = 2
LFB_length = 30
load_exist_LFB = True # False

MODEL = M9_resnest_lstm_nl_tvl() # M5_densenet_lstm_nl() /  M6_resnest_lstm_nl()


model_LFB = LFB_resnest_lstm()
# long feature bank path
LFB_test_path_save_path = "./LFB/LFB_test_resnest.pkl"




#########################################
device = "cuda:1"  
model_pure_name= 'NL-TVL-resnest' # used for the name of result pkl file
# test model path
model_path = "/media/yilin/tmrnet/models/temp/non-local/resnest_pretrained_lr1e-5_L30_2fc_mutiConv6_6-25train/latest_model_11.pth"

# lfb model path
LFB_path = "/media/yilin/tmrnet/models/temp/resnest/sl10_lr1e-5_6-25train_2/latest_model_1.pth"
###########################################


# train M4  # just example output using test01-test02
test_model(MODEL,test_dataset, test_num_each)

num test start idx :  15200
last idx test start:  15415
num of test dataset:  15425
num of test we use :  15200
num of all test use: 152000
loading features!>.........
load completed
g_LFB_test shape: (15200, 512)


/tmp/ipykernel_15977/201459335.py:153: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = Sm(outputs)


test_corrects: tensor(4, device='cuda:1')
test_corrects: tensor(4, device='cuda:1')
test_corrects: tensor(4, device='cuda:1')
test_corrects: tensor(10, device='cuda:1')
test_corrects: tensor(17, device='cuda:1')
test_corrects: tensor(17, device='cuda:1')
test_corrects: tensor(18, device='cuda:1')
test_corrects: tensor(25, device='cuda:1')
test_corrects: tensor(30, device='cuda:1')
test_corrects: tensor(34, device='cuda:1')
test_corrects: tensor(36, device='cuda:1')
test_corrects: tensor(37, device='cuda:1')
test_corrects: tensor(37, device='cuda:1')
test_corrects: tensor(37, device='cuda:1')
test_corrects: tensor(37, device='cuda:1')
test_corrects: tensor(37, device='cuda:1')
test_corrects: tensor(37, device='cuda:1')
test_corrects: tensor(37, device='cuda:1')
test_corrects: tensor(37, device='cuda:1')
test_corrects: tensor(40, device='cuda:1')
test_corrects: tensor(41, device='cuda:1')
test_corrects: tensor(51, device='cuda:1')
test_corrects: tensor(61, device='cuda:1')
test_corrects:

test_corrects: tensor(717, device='cuda:1')
test_corrects: tensor(720, device='cuda:1')
test_corrects: tensor(724, device='cuda:1')
test_corrects: tensor(728, device='cuda:1')
test_corrects: tensor(728, device='cuda:1')
test_corrects: tensor(734, device='cuda:1')
test_corrects: tensor(744, device='cuda:1')
test_corrects: tensor(753, device='cuda:1')
test_corrects: tensor(753, device='cuda:1')
test_corrects: tensor(757, device='cuda:1')
test_corrects: tensor(761, device='cuda:1')
test_corrects: tensor(770, device='cuda:1')
test_corrects: tensor(770, device='cuda:1')
test_corrects: tensor(780, device='cuda:1')
test_corrects: tensor(790, device='cuda:1')
test_corrects: tensor(800, device='cuda:1')
test_corrects: tensor(810, device='cuda:1')
test_corrects: tensor(820, device='cuda:1')
test_corrects: tensor(830, device='cuda:1')
test_corrects: tensor(840, device='cuda:1')
test_corrects: tensor(846, device='cuda:1')
test_corrects: tensor(846, device='cuda:1')
test_corrects: tensor(847, devic

test_corrects: tensor(2095, device='cuda:1')
test_corrects: tensor(2104, device='cuda:1')
test_corrects: tensor(2114, device='cuda:1')
test_corrects: tensor(2124, device='cuda:1')
test_corrects: tensor(2133, device='cuda:1')
test_corrects: tensor(2143, device='cuda:1')
test_corrects: tensor(2153, device='cuda:1')
test_corrects: tensor(2163, device='cuda:1')
test_corrects: tensor(2173, device='cuda:1')
test_corrects: tensor(2183, device='cuda:1')
test_corrects: tensor(2190, device='cuda:1')
test_corrects: tensor(2200, device='cuda:1')
test_corrects: tensor(2210, device='cuda:1')
test_corrects: tensor(2220, device='cuda:1')
test_corrects: tensor(2230, device='cuda:1')
test_corrects: tensor(2240, device='cuda:1')
test_corrects: tensor(2249, device='cuda:1')
test_corrects: tensor(2257, device='cuda:1')
test_corrects: tensor(2266, device='cuda:1')
test_corrects: tensor(2275, device='cuda:1')
test_corrects: tensor(2283, device='cuda:1')
test_corrects: tensor(2293, device='cuda:1')
test_corre

...

test_corrects: tensor(5012, device='cuda:1')
test_corrects: tensor(5022, device='cuda:1')
test_corrects: tensor(5032, device='cuda:1')
test_corrects: tensor(5042, device='cuda:1')
test_corrects: tensor(5052, device='cuda:1')
test_corrects: tensor(5060, device='cuda:1')
test_corrects: tensor(5069, device='cuda:1')
test_corrects: tensor(5079, device='cuda:1')
test_corrects: tensor(5089, device='cuda:1')
test_corrects: tensor(5099, device='cuda:1')
test_corrects: tensor(5109, device='cuda:1')
test_corrects: tensor(5119, device='cuda:1')
test_corrects: tensor(5129, device='cuda:1')
test_corrects: tensor(5139, device='cuda:1')
test_corrects: tensor(5149, device='cuda:1')
test_corrects: tensor(5158, device='cuda:1')
test_corrects: tensor(5165, device='cuda:1')
test_corrects: tensor(5174, device='cuda:1')
test_corrects: tensor(5180, device='cuda:1')
test_corrects: tensor(5190, device='cuda:1')
test_corrects: tensor(5200, device='cuda:1')
test_corrects: tensor(5209, device='cuda:1')
test_corre

test_corrects: tensor(6566, device='cuda:1')
test_corrects: tensor(6574, device='cuda:1')
test_corrects: tensor(6584, device='cuda:1')
test_corrects: tensor(6592, device='cuda:1')
test_corrects: tensor(6602, device='cuda:1')
test_corrects: tensor(6610, device='cuda:1')
test_corrects: tensor(6620, device='cuda:1')
test_corrects: tensor(6630, device='cuda:1')
test_corrects: tensor(6640, device='cuda:1')
test_corrects: tensor(6649, device='cuda:1')
test_corrects: tensor(6659, device='cuda:1')
test_corrects: tensor(6669, device='cuda:1')
test_corrects: tensor(6679, device='cuda:1')
test_corrects: tensor(6689, device='cuda:1')
test_corrects: tensor(6699, device='cuda:1')
test_corrects: tensor(6709, device='cuda:1')
test_corrects: tensor(6718, device='cuda:1')
test_corrects: tensor(6728, device='cuda:1')
test_corrects: tensor(6738, device='cuda:1')
test_corrects: tensor(6747, device='cuda:1')
test_corrects: tensor(6754, device='cuda:1')
test_corrects: tensor(6763, device='cuda:1')
test_corre

test_corrects: tensor(7995, device='cuda:1')
test_corrects: tensor(8005, device='cuda:1')
test_corrects: tensor(8015, device='cuda:1')
test_corrects: tensor(8025, device='cuda:1')
test_corrects: tensor(8035, device='cuda:1')
test_corrects: tensor(8045, device='cuda:1')
test_corrects: tensor(8055, device='cuda:1')
test_corrects: tensor(8065, device='cuda:1')
test_corrects: tensor(8073, device='cuda:1')
test_corrects: tensor(8083, device='cuda:1')
test_corrects: tensor(8093, device='cuda:1')
test_corrects: tensor(8103, device='cuda:1')
test_corrects: tensor(8113, device='cuda:1')
test_corrects: tensor(8121, device='cuda:1')
test_corrects: tensor(8129, device='cuda:1')
test_corrects: tensor(8138, device='cuda:1')
test_corrects: tensor(8148, device='cuda:1')
test_corrects: tensor(8158, device='cuda:1')
test_corrects: tensor(8168, device='cuda:1')
test_corrects: tensor(8178, device='cuda:1')
test_corrects: tensor(8188, device='cuda:1')
test_corrects: tensor(8198, device='cuda:1')
test_corre

test_corrects: tensor(9067, device='cuda:1')
test_corrects: tensor(9076, device='cuda:1')
test_corrects: tensor(9086, device='cuda:1')
test_corrects: tensor(9095, device='cuda:1')
test_corrects: tensor(9105, device='cuda:1')
test_corrects: tensor(9115, device='cuda:1')
test_corrects: tensor(9124, device='cuda:1')
test_corrects: tensor(9129, device='cuda:1')
test_corrects: tensor(9138, device='cuda:1')
test_corrects: tensor(9143, device='cuda:1')
test_corrects: tensor(9153, device='cuda:1')
test_corrects: tensor(9163, device='cuda:1')
test_corrects: tensor(9172, device='cuda:1')
test_corrects: tensor(9178, device='cuda:1')
test_corrects: tensor(9188, device='cuda:1')
test_corrects: tensor(9198, device='cuda:1')
test_corrects: tensor(9208, device='cuda:1')
test_corrects: tensor(9218, device='cuda:1')
test_corrects: tensor(9228, device='cuda:1')
test_corrects: tensor(9235, device='cuda:1')
test_corrects: tensor(9243, device='cuda:1')
test_corrects: tensor(9243, device='cuda:1')
test_corre

test_corrects: tensor(10606, device='cuda:1')
test_corrects: tensor(10610, device='cuda:1')
test_corrects: tensor(10620, device='cuda:1')
test_corrects: tensor(10630, device='cuda:1')
test_corrects: tensor(10632, device='cuda:1')
test_corrects: tensor(10642, device='cuda:1')
test_corrects: tensor(10652, device='cuda:1')
test_corrects: tensor(10662, device='cuda:1')
test_corrects: tensor(10672, device='cuda:1')
test_corrects: tensor(10682, device='cuda:1')
test_corrects: tensor(10692, device='cuda:1')
test_corrects: tensor(10702, device='cuda:1')
test_corrects: tensor(10711, device='cuda:1')
test_corrects: tensor(10721, device='cuda:1')
test_corrects: tensor(10731, device='cuda:1')
test_corrects: tensor(10741, device='cuda:1')
test_corrects: tensor(10751, device='cuda:1')
test_corrects: tensor(10761, device='cuda:1')
test_corrects: tensor(10771, device='cuda:1')
test_corrects: tensor(10781, device='cuda:1')
test_corrects: tensor(10791, device='cuda:1')
test_corrects: tensor(10799, devic